In [35]:
#class pcb declared with definition function to initialize all pcb functions for phase 3
class PCB:
    def __init__(self, p_id, cpu_state, memory, scheduling_info, accounting_info, process_state,
                 parent, children, open_files, other_resources, arrival_time, cpu_required, quantum, context_switch_penalty):
        self.p_id = p_id
        self.cpu_state = cpu_state
        self.memory = memory
        self.scheduling_info = scheduling_info
        self.accounting_info = accounting_info
        self.process_state = process_state
        self.parent = parent
        self.children = children
        self.open_files = open_files
        self.other_resources = other_resources
        self.arrival_time = arrival_time
        self.cpu_required = cpu_required
        self.remaining_time = cpu_required  # New: Tracks remaining CPU time
        self.quantum = quantum
        self.context_switch_penalty = context_switch_penalty
        self.start_time = None
        self.finish_time = None
#returns initialized variables
    def __str__(self):
        return f"{self.p_id} {self.cpu_state} {self.memory} {self.scheduling_info} {self.accounting_info} " \
               f"{self.process_state} {self.parent} {self.children} {self.open_files} {self.other_resources} " \
               f"{self.arrival_time} {self.cpu_required} {self.quantum} {self.context_switch_penalty}"
# reads the phase 3 test data if length of row == 14, splitting each index in the line. With error handling
#if no file found with try: and except


def read_pcb_file(filename):
    pcbs = []
    try:
        with open(filename, 'r') as file:
            for line in file:
                data = line.strip().split()
                if len(data) == 14:
                    pcb = PCB(
                        p_id=int(data[0]),
                        cpu_state=int(data[1]),
                        memory=int(data[2]),
                        scheduling_info=int(data[3]),
                        accounting_info=int(data[4]),
                        process_state=data[5],
                        parent=None if data[6] == 'NULL' else int(data[6]),
                        children=None if data[7] == 'NULL' else int(data[7]),
                        open_files=None if data[8] == 'NULL' else int(data[8]),
                        other_resources=data[9],
                        arrival_time=int(data[10]),
                        cpu_required=int(data[11]),
                        quantum=int(data[12]),
                        context_switch_penalty=int(data[13])
                    )
                    pcbs.append(pcb)
    except FileNotFoundError:
        print(f"File {filename} not found.")
    return pcbs
# adds the functions required from the test data and in this case especially the quantum size needed for RR
def add_pcb(pcbs):
    try:
        p_id = int(input("Enter process ID: "))
        cpu_state = int(input("Enter CPU state: "))
        memory = int(input("Enter memory needed: "))
        scheduling_info = int(input("Enter scheduling information: "))
        accounting_info = int(input("Enter accounting information: "))
        process_state = input("Enter process state: ")
        parent = input("Enter parent process ID (or NULL): ")
        children = input("Enter child process ID (or NULL): ")
        open_files = input("Enter open file handler (or NULL): ")
        other_resources = input("Enter other resources description: ")
        arrival_time = int(input("Enter arrival time: "))
        cpu_required = int(input("Enter CPU required: "))
        quantum = int(input("Enter quantum size: "))
        context_switch_penalty = int(input("Enter context switch penalty: "))

        parent = None if parent == 'NULL' else int(parent)
        children = None if children == 'NULL' else int(children)
        open_files = None if open_files == 'NULL' else int(open_files)

        pcb = PCB(p_id, cpu_state, memory, scheduling_info, accounting_info, process_state,
                  parent, children, open_files, other_resources, arrival_time, cpu_required,
                  quantum, context_switch_penalty)
        pcbs.append(pcb)
        print("PCB added successfully.")
    except ValueError:
        print("Invalid input. Please enter correct values.")

def display_pcbs(pcbs):
    if not pcbs:
        print("No PCBs available to display.")
    for pcb in pcbs:
        print(pcb)
#write new process control blocks into the test data file using try: if invalid returns error message
def save_pcbs_to_file(pcbs, filename):
    try:
        with open(filename, 'w') as file:
            for pcb in pcbs:
                file.write(f"{pcb.p_id} {pcb.cpu_state} {pcb.memory} {pcb.scheduling_info} "
                           f"{pcb.accounting_info} {pcb.process_state} "
                           f"{'NULL' if pcb.parent is None else pcb.parent} "
                           f"{'NULL' if pcb.children is None else pcb.children} "
                           f"{'NULL' if pcb.open_files is None else pcb.open_files} "
                           f"{pcb.other_resources} {pcb.arrival_time} {pcb.cpu_required} "
                           f"{pcb.quantum} {pcb.context_switch_penalty}\n")
        print(f"PCBs saved to {filename}.")
    except IOError:
        print(f"Error saving to file {filename}.")

def fcfs_scheduler(pcbs):
    # Sort processes by arrival time
    pcbs.sort(key=lambda x: x.arrival_time)
    current_time = 0
    total_turnaround_time = 0
    context_switches = 0
    process_execution_order = ""
    time_slice = 3
    completed_processes = 0  # Track how many processes are completed
    print("\nRunning FCFS Scheduler:\n")
    
    for process in pcbs:
        if current_time < process.arrival_time:
            current_time = process.arrival_time  # If idle time exists, fast forward to the next arrival
        process.css_time = current_time
        process.cse_time = current_time + process.cpu_required
        current_time = process.cse_time
        turnaround_time = process.cse_time - process.arrival_time
        run_time = min(process.remaining_time, time_slice)
        for i in range(run_time):
            process_execution_order += f"P{process.p_id}"
        total_turnaround_time += turnaround_time
        completed_processes += 1  # Increment the count of completed processes
        if context_switches < 3:
            context_switches += 1 #increment the count of context switches
        print(f"Process {process.p_id} - Start: {process.css_time}, Finish: {process.cse_time}, Turnaround: {turnaround_time}")
    
    # Only calculate average turnaround time if we have completed any processes
    if completed_processes > 0:
        average_turnaround_time = total_turnaround_time / completed_processes
        print(f"\nAverage Turnaround Time for FCFS: {average_turnaround_time:.2f}\n")
    else:
        process_execution_order += "_"
        print("\nNo processes were scheduled.\n")
    print(f"\nTotal Context Switches: {context_switches}")
    print(f"Execution Order: {process_execution_order}")


def sjf_scheduler(pcbs):
    # Sort processes first by arrival time, then by CPU required
    pcbs.sort(key=lambda x: (x.arrival_time, x.cpu_required))
    current_time = 0
    total_turnaround_time = 0
    context_switches = 0
    process_execution_order = ""
    time_slice = 3
    ready_queue = []
    completed_processes = 0  # Track how many processes are completed
    print("\nRunning SJF Scheduler:\n")
    
    while pcbs or ready_queue:
        while pcbs and pcbs[0].arrival_time <= current_time:
            ready_queue.append(pcbs.pop(0))
        if ready_queue:
            ready_queue.sort(key=lambda x: x.cpu_required)  # Sort by CPU required
            process = ready_queue.pop(0)
            process.start_time = current_time
            process.finish_time = current_time + process.cpu_required
            current_time = process.finish_time
            turnaround_time = process.finish_time - process.arrival_time
            run_time = min(process.remaining_time, time_slice)
            for i in range(run_time):
                process_execution_order += f"P{process.p_id}"
            total_turnaround_time += turnaround_time
            completed_processes += 1  # Increment the count of completed processes
            if context_switches < 3:
                context_switches += 1 #increment the count of context switches
            print(f"Process {process.p_id} - Start: {process.start_time}, Finish: {process.finish_time}, Turnaround: {turnaround_time}")
        else:
            current_time += 1  # No process ready, increment time
    
    # Only calculate average turnaround time if we have completed any processes
    if completed_processes > 0:
        average_turnaround_time = total_turnaround_time / completed_processes
        print(f"\nAverage Turnaround Time for SJF: {average_turnaround_time:.2f}\n")
    else:
        process_execution_order += "_"
        print("\nNo processes were scheduled.\n")
    print(f"\nTotal Context Switches: {context_switches}")
    print(f"Execution Order: {process_execution_order}")

def round_robin_scheduler(pcbs, quantum):
    current_time = 0
    context_switches = 0
    ready_queue = []
    process_execution_order = ""
    total_turnaround_time = 0
    completed_processes = 0
    time_slice = quantum

    print("\nRunning Round-Robin Scheduler:\n")
    
    # Initialize a context switch counter for each process
    for process in pcbs:
        process.context_switches = 0  # Track individual process switches
    
    while pcbs or ready_queue:
        # Add processes that have arrived by the current time to the ready queue
        while pcbs and pcbs[0].arrival_time <= current_time:
            ready_queue.append(pcbs.pop(0))

        # Flag to track if a round has been completed
        round_completed = True

        for process in ready_queue:
            if process.remaining_time > 0 and process.context_switches < 3:
                round_completed = False
                break

        # Reset the context switch counter for all processes if the round is completed
        if round_completed:
            for process in ready_queue:
                process.context_switches = 0

        if ready_queue:
            process = ready_queue.pop(0)
            
            # Check if process has reached its maximum context switches for the round
            if process.context_switches < 3:
                # Determine how much time the process will run during this round
                run_time = min(process.remaining_time, time_slice)
                
                # Execute the process for the determined runtime
                process_execution_order += f"P{process.p_id}-"  # Add a separator for each context switch
                current_time += run_time
                process.remaining_time -= run_time
                process.context_switches += 1  # Increment the process's context switches
                context_switches += 1  # Increment total context switches

                # If the process has finished
                if process.remaining_time == 0:
                    process.finish_time = current_time
                    turnaround_time = process.finish_time - process.arrival_time
                    total_turnaround_time += turnaround_time
                    completed_processes += 1
                    print(f"Process {process.p_id} completed at clock cycle {current_time}, Turnaround time: {turnaround_time}")
                else:
                    ready_queue.append(process)  # Put it back in the queue if not finished

            else:
                # Process has reached its max context switches for the round; requeue without incrementing time
                print(f"Process {process.p_id} has reached its context switch limit for this round.")
                ready_queue.append(process)  # Move it to the end of the queue

        else:
            process_execution_order += "_-"  # Indicate idle time with a separator
            current_time += 1  # No process available, increment time

    # Remove the trailing separator for a cleaner output
    process_execution_order = process_execution_order.rstrip("-")

    # Average Turnaround Time
    if completed_processes > 0:
        average_turnaround_time = total_turnaround_time / completed_processes
        print(f"\nAverage Turnaround Time: {average_turnaround_time:.2f}")
    else:
        print("\nNo processes were scheduled.\n")

    print(f"\nTotal Context Switches: {context_switches}")
    print(f"Execution Order: {process_execution_order}")


#main function to execute UI 
def main():
    #reads in test data file
    filename = 'Project Phase 3 Official Test Data File.txt'
    pcbs = read_pcb_file(filename)
    #user given a choice that while true, able to run all 3 algorithms with necessary requirementssuch as
    #turnaround time, avg turnaround time, context switches, process execution in a string, quantum, etc.
    while True:
        print("\nScheduler Menu:")
        print("1. Run FCFS Scheduler")
        print("2. Run SJF Scheduler")
        print("3. Run Round-Robin Scheduler")
        print("4. Exit Scheduler")
        choice = input("Enter your choice: ")
        
        if choice == '1':
            fcfs_scheduler(pcbs)
        elif choice == '2':
            sjf_scheduler(pcbs)
        elif choice == '3':
            quantum = int(input("Enter the quantum size for Round-Robin: "))
            round_robin_scheduler(pcbs, quantum)
        elif choice == '4':
            print("Exiting scheduler...")
            break
        else:
            print("Invalid choice. Please select a valid option.")
if __name__ == "__main__":
    main()


Scheduler Menu:
1. Run FCFS Scheduler
2. Run SJF Scheduler
3. Run Round-Robin Scheduler
4. Exit Scheduler


Enter your choice:  1



Running FCFS Scheduler:

Process 0 - Start: 0, Finish: 8, Turnaround: 8
Process 1 - Start: 8, Finish: 16, Turnaround: 15
Process 2 - Start: 16, Finish: 17, Turnaround: 15
Process 3 - Start: 17, Finish: 20, Turnaround: 17
Process 4 - Start: 20, Finish: 24, Turnaround: 21
Process 5 - Start: 24, Finish: 28, Turnaround: 24

Average Turnaround Time for FCFS: 16.67


Total Context Switches: 3
Execution Order: P0P0P0P1P1P1P2P3P3P3P4P4P4P5P5P5

Scheduler Menu:
1. Run FCFS Scheduler
2. Run SJF Scheduler
3. Run Round-Robin Scheduler
4. Exit Scheduler


Enter your choice:  2



Running SJF Scheduler:

Process 0 - Start: 0, Finish: 8, Turnaround: 8
Process 2 - Start: 8, Finish: 9, Turnaround: 7
Process 3 - Start: 9, Finish: 12, Turnaround: 9
Process 4 - Start: 12, Finish: 16, Turnaround: 13
Process 5 - Start: 16, Finish: 20, Turnaround: 16
Process 1 - Start: 20, Finish: 28, Turnaround: 27

Average Turnaround Time for SJF: 13.33


Total Context Switches: 3
Execution Order: P0P0P0P2P3P3P3P4P4P4P5P5P5P1P1P1

Scheduler Menu:
1. Run FCFS Scheduler
2. Run SJF Scheduler
3. Run Round-Robin Scheduler
4. Exit Scheduler


Enter your choice:  4


Exiting scheduler...
